### 2a
* 3 lagen 1 input 1 hidden 1 output.
    * 3 input neurons vanwegen 3 features
    * vanwegen de duimregel mean van input en output. en de duimregel even getallen. pakken we 4 neurons in hidden layer.
    * 6 output vanwege 6 verschillende klasse
    * vanwege simpele data set zijn meer hidden layers niet nodig.

* activation
    * eindigen op softmax
    * relu voor andere lagen.
* cost function
    * categorial cross entropy
* optimizer
    * Adam
* numbers of epoch.
    * epochs zijn in het begin ingesteld op 15.
* Batch size
    * 20/80 split voor test en train
    * train
        * delen in 4. batches van 100
* Learning rate
    * 0.05


In [1]:
!pip install optuna


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('bmi.csv', sep=',')
df = df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
def convert_sex(sex):
    if(sex == 'Male'):
        return 1
    else:
        return 0
df['Gender'] = df['Gender'].apply(convert_sex)


X = df.drop(columns=['Index'])
y = df['Index']  # Target column

df.head()

2024-09-30 13:01:51.263241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 13:01:51.404714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-09-30 13:01:51.404762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-09-30 13:01:51.435689: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 13:01:52.134182: W tensorflow/stream_executor/platform/de

,Gender,Height,Weight,Index
0,0,171,120,5
1,0,164,160,5
2,0,186,146,5
3,0,181,106,4
4,0,198,50,0


In [3]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df['Index'], random_state=42)

print(df.dtypes)  # Controleer de types van je kolommen

y_train = to_categorical(y_train, num_classes=6)  # Assuming 6 classes
y_test = to_categorical(y_test, num_classes=6)

print(X_train.shape)
print(y_train.shape)


Gender    int64
Height    int64
Weight    int64
Index     int64
dtype: object
(400, 3)
(400, 6)


In [4]:
model = Sequential()
model.add(Dense(4, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(6, activation='softmax'))

from keras.metrics import Recall
from tensorflow.keras.metrics import AUC

model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=[AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)



Epoch 1/150
2024-09-30 13:01:53.503956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-09-30 13:01:53.503993: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-09-30 13:01:53.504013: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-70241bed-abd0-479d-8376-545af7db2871): /proc/driver/nvidia/version does not exist
2024-09-30 13:01:53.504253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
4/4 [==============================] - 1s 79ms/step - loss: 57.9031 - au

In [5]:

from sklearn.metrics import confusion_matrix
# 1. Get predictions (probabilities) from the model
y_pred = model.predict(X_test)

# 2. Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the index of the highest probability for each sample

# 3. Convert true labels (y_test) from one-hot encoded to class labels
y_test_classes = np.argmax(y_test, axis=1)

# 4. Now, compute the confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

print(cm)


4/4 [==============================] - 0s 1ms/step
[[ 0  0  0  0  0  3]
 [ 0  0  0  0  0  4]
 [ 0  0  0  0  0 14]
 [ 0  0  0  0  0 14]
 [ 0  0  0  0  0 26]
 [ 0  0  0  0  0 39]]


### Eerste Run


model.compile(Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=100, epochs=15, validation_split=0.1)

15 epochs 864ms

De confusion matrix zet alles in klasse 6

Conclusie: loss nog vrij hoog accuracy nog vrij laag, dus we zouden hem graag wat langer willen laten trainen, en de confusion matrix liet zien dat de accuracy hoogst was wanneer alles in klasse 6 werdt gezet. Dit betekend dat de model evaluation verkeerd niet alleen op accuracy zou moeten.

Tweede Run

model.compile(Adam(learning_rate=0.05), loss='categorical_crossentropy', metrics=['accuracy', AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)

5s 929ms

De confusion matrix voorspelt in de laatste 3 klassen 1 klas omhoog en omlaag

Vanaf epoch 110 ongeveer stabiel blijft

Conclusie: Hij doet het al beter dan de eerste, dus de evaluation metrics zijn vrij goed. vanaf epoch 110 blijft hij vrij stabiel alleen springt de accuracy de hele tijd een beetje omhoog en omlaag, dus wij dachten dat de learning rate verlagen dit misschien wel op zou lossen.

Derde Run

model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=[AUC()])
model.fit(X_train, y_train, batch_size=100, epochs=150, validation_split=0.1)

5s 902ms (vrij snel nog)

De confusion matrix ziet er niet heel slecht uit, sommige zet hij nog wel verkeerd

[[ 3  0  0  0  0  0]
 [ 0  4  0  0  0  0]
 [ 0  3  9  1  1  0]
 [ 0  0  2  6  6  0]
 [ 0  0  0  4 19  3]
 [ 0  0  0  0  6 33]]

met een lage learning rate flatlined de AUC rond de 120 epochs dus meer dan het huidige hebben we niet nodig.

Conclusie: Dit zijn de waardes waarmee we het beste resultaat verkrijgen. We merken alleen wel dat de dataset heel erg naar de laatste klasse leunt waardoor het NN af en toe alles in klassen 6 wilt zetten.

Optuna:

In [6]:
import optuna

# Define optuna objective function
def objective(trial):
    # Define the hyperparameters to optimize
    num_layers = trial.suggest_int('num_layers', 1, 3)
    num_neurons = trial.suggest_int('num_neurons', 8, 64)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_int('batch_size', 8, 32)

    # Create the model
    model = Sequential()
    model.add(Dense(4, input_shape=(X_train.shape[1],), activation='relu'))

    for i in range(num_layers):
        model.add(Dense(num_neurons, activation='relu'))

    model.add(Dense(6, activation='softmax'))

    # Compile the model
    model.compile(Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=[AUC()])

    # Fit the model
    model.fit(X_train, y_train, batch_size=batch_size, epochs=50, validation_split=0.1)

    # Evaluate the model
    _, auc = model.evaluate(X_test, y_test, verbose=0)

    return auc

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

18/18 [==============================] - 0s 2ms/step - loss: 0.6241 - auc_91: 0.9569 - val_loss: 0.8960 - val_auc_91: 0.9243
Epoch 16/50
18/18 [==============================] - 0s 4ms/step - loss: 0.6240 - auc_91: 0.9563 - val_loss: 0.8216 - val_auc_91: 0.9332
Epoch 17/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6110 - auc_91: 0.9593 - val_loss: 0.8800 - val_auc_91: 0.9280
Epoch 18/50
18/18 [==============================] - 0s 4ms/step - loss: 0.6215 - auc_91: 0.9570 - val_loss: 0.7172 - val_auc_91: 0.9443
Epoch 19/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5976 - auc_91: 0.9584 - val_loss: 0.7442 - val_auc_91: 0.9352
Epoch 20/50
18/18 [==============================] - 0s 4ms/step - loss: 0.6235 - auc_91: 0.9562 - val_loss: 0.7256 - val_auc_91: 0.9423
Epoch 21/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6227 - auc_91: 0.9568 - val_loss: 0.8689 - val_auc_91: 0.9277
Epoch 22/50
18/18 [==============================] - 

In [11]:
print(study.best_trial.params)

{'num_layers': 3, 'num_neurons': 45, 'learning_rate': 0.005704347102265108, 'batch_size': 28}


De beste uitkomst is: 3 layers, 45 neurons per layer, learning rate van 0.057, batch size van 28

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=70241bed-abd0-479d-8376-545af7db2871' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>